# Assign country codes to affiliations

References:

- https://github.com/greenelab/iscb-diversity/issues/8
- https://towardsdatascience.com/geoparsing-with-python-c8f4c9f78940
- https://github.com/elyase/geotext/issues/23

In [1]:
import pathlib
import functools
import re
import lzma

import backoff
import geotext
import geopy.geocoders
import jsonlines
import pandas
import ratelimit
import tqdm.notebook

In [2]:
# read PubMed Central affiliations
affil_pmc_df = pandas.read_csv("data/pmc/affiliations.tsv.xz", sep='\t')
assert affil_pmc_df.affiliation.notna().all()
affil_pmc_df.head(2)

,pmcid,position,affiliation
0,PMC100321,1,"1 University of Cologne, Institute of Genetics..."
1,PMC100321,2,"1 University of Cologne, Institute of Genetics..."


In [3]:
# read PubMed affiliations
affil_pm_df = pandas.read_csv("data/pubmed/affiliations.tsv.xz", sep='\t')
assert affil_pm_df.affiliation.notna().all()
affil_pm_df.head(2)

,pmid,position,affiliation
0,7477412,1,"Dept. of Pathology, Cornell Medical College, N..."
1,7479891,1,"National Center for Human Genome Research, Nat..."


In [4]:
# combine affiliations from PubMed Central and PubMed
affiliations = sorted(set(affil_pmc_df.affiliation) | set(affil_pm_df.affiliation))
print(f"{len(affiliations):,} unique affiliation strings")
affiliations[:5]

439,400 unique affiliation strings


['"ASKION" GmbH, Gewerbepark Keplerstraße 17-19, 07549, Gera, Germany.',
 '"Athena" Research and Innovation Center, Athens, 15125, Greece.',
 '"Athena" Research and Innovation Center, Athens, 15125, Greece. kzagganas@uop.gr.',
 '"Biology of Spirochetes" Unit, Institut Pasteur, 28 Rue Du Docteur Roux, 75724, Paris Cedex 15, France, mathieu.picardeau@pasteur.fr.',
 '"Cephalogenetics" Genetic Center, Athens, Greece.']

In [5]:
def get_countries_geotext(text):
    """
    Use geotext package to get country metions.
    The returned counts dict maps from country code
    to number of places located to that county in the text.

    For example, the following code detects GR twice, first for
    "Athens" and second for "Greece":

    ```
    >>> text = '"Athena" Research and Innovation Center, Athens, 15125, Greece.'
    >>> get_countries_geotext(text)
    {'GR': 2}
    ```

    See https://github.com/elyase/geotext
    """
    geo_text = geotext.GeoText(text)
    return dict(geo_text.country_mentions)

geopy.geocoders.options.default_user_agent = 'https://github.com/greenelab/iscb-diversity'
geolocator = geopy.geocoders.Nominatim(timeout=5)

@functools.lru_cache(maxsize=200_000)
@backoff.on_exception(backoff.expo, ratelimit.RateLimitException, max_tries=8)
@ratelimit.limits(calls=1, period=1)
def _geocode(text):
    """
    https://operations.osmfoundation.org/policies/nominatim/
    """
    return geolocator.geocode(text, addressdetails=True)

email_pattern = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
split_pattern = re.compile(r"; |, |\. ")

def get_country_geocode(affiliation):
    """
    Lookup country using OSM Nomatim.
    Splits affiliations text into parts by punctuation,
    searching for country matches in reverse. Parts that
    look like email addresses are discarded.
    """
    parts = split_pattern.split(affiliation)
    for part in reversed(parts):
        part = part.rstrip(".")
        if email_pattern.fullmatch(part):
            continue
        location = _geocode(part)
        if not location:
            continue
        try:
            return location.raw['address']['country_code'].upper()
        except KeyError:
            return None


def query_affiliation(affiliation: str):
    return dict(
        affiliation=affiliation,
        country_geocode=get_country_geocode(affiliation),
        countries_geotext=get_countries_geotext(affiliation),
    )

In [6]:
# read already queried affiliations
path = pathlib.Path('data/affiliations/geocode.jsonl.xz')
lines = jsonlines.Reader(lzma.open(path, "rt")) if path.exists() else []
existing = {row['affiliation'] for row in lines}
new = sorted(set(affiliations) - existing)
print(f"{len(affiliations):,} total affiliations: {len(existing):,} already queried, {len(new):,} new")

439,400 total affiliations: 464,780 already queried, 0 new


In [7]:
# query new affiliations and append to JSON Lines file
with lzma.open(path, mode='at') as write_file:
    with jsonlines.Writer(write_file) as writer:
        for affiliation in tqdm.notebook.tqdm(new):
            result = query_affiliation(affiliation)
            writer.write(result)

In [8]:
# Read the jsonlines file
with jsonlines.Reader(lzma.open(path, "rt")) as reader:
    lines = list(reader)
# Show a single line
lines[10]

{'affiliation': "'Multimodal Computing and Interaction', Saarland University & Department for Computational Biology and Applied Computing, Max Planck Institute for Informatics, Saarbrücken, 66123 Saarland, Germany, Ray and Stephanie Lane Center for Computational Biology, Carnegie Mellon University, Pittsburgh, 15206 PA, USA, Department of Mathematics and Computer Science, Freie Universität Berlin, 14195 Berlin, Germany, Université Pierre et Marie Curie, UMR7238, CNRS-UPMC, Paris, France and CNRS, UMR7238, Laboratory of Computational and Quantitative Biology, Paris, France.",
 'country_geocode': 'FR',
 'countries_geotext': {'DE': 4, 'FR': 4, 'US': 2},
 'countries': ['DE', 'FR', 'US']}

In [9]:
def get_consensus_countries(line: dict) -> list:
    """
    Get a list of countries resulting from a consensus algorithm
    between countries_geotext and country_geocode.
    """
    country_geotext: dict = line['countries_geotext']
    country_geocode: str = line['country_geocode']
    if not country_geotext:
        # geotext empty, so use geocode country or nothing
        return [country_geocode] if country_geocode else []
    countries_gte_2 = {
        country: count for country, count in
        country_geotext.items() if count >= 2
    }
    if countries_gte_2:
        # countries with multiple mentions according to geotext
        return list(countries_gte_2)
    if country_geocode and country_geocode in country_geotext:
        # geocode country matches a geotext country
        return [country_geocode]
    return list(country_geotext)

In [10]:
for line in lines:
    line["countries"] = get_consensus_countries(line)

In [11]:
lines[10]

{'affiliation': "'Multimodal Computing and Interaction', Saarland University & Department for Computational Biology and Applied Computing, Max Planck Institute for Informatics, Saarbrücken, 66123 Saarland, Germany, Ray and Stephanie Lane Center for Computational Biology, Carnegie Mellon University, Pittsburgh, 15206 PA, USA, Department of Mathematics and Computer Science, Freie Universität Berlin, 14195 Berlin, Germany, Université Pierre et Marie Curie, UMR7238, CNRS-UPMC, Paris, France and CNRS, UMR7238, Laboratory of Computational and Quantitative Biology, Paris, France.",
 'country_geocode': 'FR',
 'countries_geotext': {'DE': 4, 'FR': 4, 'US': 2},
 'countries': ['DE', 'FR', 'US']}

In [12]:
with lzma.open(path, mode='wt') as write_file:
    with jsonlines.Writer(write_file) as writer:
        writer.write_all(lines)

In [13]:
affil_df = pandas.DataFrame(lines)
affil_df = (
    affil_df
    .explode("countries")
    .rename(columns={"countries": "country"})
    .dropna(subset=['country'])
    [["affiliation", "country"]]
    .drop_duplicates()  # for safety, duplicates not expected
)
affil_df.head(2)

,affiliation,country
0,"""Athena"" Research and Innovation Center, Athen...",GR
1,"""Athena"" Research and Innovation Center, Athen...",GR


In [14]:
# Most common countries
affil_df.country.value_counts(normalize=True).head(20).map("{:.02%}".format)

US    30.79%
CN    12.93%
GB     6.91%
DE     6.09%
FR     3.94%
CA     3.12%
IT     3.10%
JP     2.89%
ES     2.82%
AU     2.62%
NL     1.88%
IN     1.69%
KR     1.65%
BR     1.52%
CH     1.32%
SE     1.25%
TW     1.11%
BE     1.01%
DK     0.96%
SG     0.69%
Name: country, dtype: object

In [15]:
# save affiliations to a table
affil_df.to_csv("data/affiliations/countries.tsv.xz", sep='\t', index=False)